In [1]:
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os
os.chdir("C:\\Users\\Basudev\\Downloads\\LSTM")
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Using TensorFlow backend.
C:\Users\Basudev\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Basudev\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Basudev\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Basudev\anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
np.random.seed(1234)

In [3]:
dataset = pd.read_csv('international-airline-passengers.csv', usecols = [1], engine = 'python', skipfooter = 3)

In [4]:
dataset.head()

,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
0,112
1,118
2,132
3,129
4,121


In [5]:
dataset = dataset.values
dataset = dataset.astype('float32')

In [6]:
scaler = MinMaxScaler(feature_range = (0, 1))
dataset = scaler.fit_transform(dataset)
dataset

array([[0.01544401],
       [0.02702703],
       [0.05405405],
       [0.04826255],
       [0.03281853],
       [0.05984557],
       [0.08494207],
       [0.08494207],
       [0.06177607],
       [0.02895753],
       [0.        ],
       [0.02702703],
       [0.02123553],
       [0.04247104],
       [0.07142857],
       [0.05984557],
       [0.04054055],
       [0.08687258],
       [0.12741312],
       [0.12741312],
       [0.10424709],
       [0.05598456],
       [0.01930502],
       [0.06949806],
       [0.07915059],
       [0.08880308],
       [0.14285713],
       [0.11389962],
       [0.13127413],
       [0.14285713],
       [0.18339768],
       [0.18339768],
       [0.15444016],
       [0.11196911],
       [0.08108109],
       [0.1196911 ],
       [0.12934363],
       [0.14671814],
       [0.17181468],
       [0.14864865],
       [0.15250966],
       [0.22007722],
       [0.24324325],
       [0.26640925],
       [0.2027027 ],
       [0.16795367],
       [0.13127413],
       [0.173

In [7]:
train_size = int(len(dataset)*0.67)
print(train_size)
test_size = len(dataset) - train_size
print(test_size)
train, test = dataset[0 : train_size, :], dataset[train_size : len(dataset), :]
print(len(train), len(test))

96
48
96 48


In [8]:
def create_dataset(dataset, look_back = 1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i : (i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [9]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX)
print(testX)

[[[0.01544401]]

 [[0.02702703]]

 [[0.05405405]]

 [[0.04826255]]

 [[0.03281853]]

 [[0.05984557]]

 [[0.08494207]]

 [[0.08494207]]

 [[0.06177607]]

 [[0.02895753]]

 [[0.        ]]

 [[0.02702703]]

 [[0.02123553]]

 [[0.04247104]]

 [[0.07142857]]

 [[0.05984557]]

 [[0.04054055]]

 [[0.08687258]]

 [[0.12741312]]

 [[0.12741312]]

 [[0.10424709]]

 [[0.05598456]]

 [[0.01930502]]

 [[0.06949806]]

 [[0.07915059]]

 [[0.08880308]]

 [[0.14285713]]

 [[0.11389962]]

 [[0.13127413]]

 [[0.14285713]]

 [[0.18339768]]

 [[0.18339768]]

 [[0.15444016]]

 [[0.11196911]]

 [[0.08108109]]

 [[0.1196911 ]]

 [[0.12934363]]

 [[0.14671814]]

 [[0.17181468]]

 [[0.14864865]]

 [[0.15250966]]

 [[0.22007722]]

 [[0.24324325]]

 [[0.26640925]]

 [[0.2027027 ]]

 [[0.16795367]]

 [[0.13127413]]

 [[0.17374519]]

 [[0.17760617]]

 [[0.17760617]]

 [[0.25482625]]

 [[0.25289574]]

 [[0.24131274]]

 [[0.26833975]]

 [[0.3088803 ]]

 [[0.32432434]]

 [[0.25675675]]

 [[0.20656371]]

 [[0.14671814]

In [10]:
#LSTM(RNN) Model
model = Sequential()
model.add(LSTM(4, input_shape = (1, look_back)))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(trainX, trainY, batch_size = 1, verbose = 2)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
print(trainPredict)
print(testPredict)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
 - 1s - loss: 0.0329
[[0.10235997]
 [0.10424852]
 [0.10865176]
 [0.10770871]
 [0.10519251]
 [0.10959449]
 [0.11367524]
 [0.11367524]
 [0.10990867]
 [0.10456321]
 [0.09984111]
 [0.10424852]
 [0.10330433]
 [0.10676535]
 [0.11147888]
 [0.10959449]
 [0.10645084]
 [0.1139888 ]
 [0.12055959]
 [0.12055959]
 [0.11680833]
 [0.10896605]
 [0.10298955]
 [0.11116492]
 [0.11273424]
 [0.11430231]
 [0.1230545 ]
 [0.11837256]
 [0.12118378]
 [0.1230545 ]
 [0.1295769 ]
 [0.1295769 ]
 [0.12492222]
 [0.11805984]
 [0.11304796]
 [0.11931027]
 [0.12087172]
 [0.12367742]
 [0.12771766]
 [0.12398876]
 [0.12461114]
 [0.13543871]
 [0.13911848]
 [0.14277898]
 [0.13266718]
 [0.12709711]
 [0.12118378]
 [0.12802778]
 [0.12864774]
 [0.12864774]
 [0.14095125]
 [0.14064613]
 [0.13881254]
 [0.1430831 ]
 [0.14943454]
 [0.15183553]
 [0.14125621]
 [0.1332839 ]
 [0.12367742]
 [0.13019583]
 [0.13112

In [11]:
#Reverse Predicted Values to Actual Values
trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)
trainY = scaler.inverse_transform([trainY])
testY = scaler.inverse_transform([testY])
print(trainPredict, trainY)
print(testPredict, testY)

[[157.02248]
 [158.00073]
 [160.28162]
 [159.79312]
 [158.48972]
 [160.76994]
 [162.88377]
 [162.88377]
 [160.9327 ]
 [158.16374]
 [155.7177 ]
 [158.00073]
 [157.51164]
 [159.30446]
 [161.74606]
 [160.76994]
 [159.14153]
 [163.04619]
 [166.44987]
 [166.44987]
 [164.50671]
 [160.44441]
 [157.34859]
 [161.58344]
 [162.39633]
 [163.2086 ]
 [167.74223]
 [165.31699]
 [166.7732 ]
 [167.74223]
 [171.12083]
 [171.12083]
 [168.7097 ]
 [165.155  ]
 [162.55884]
 [165.80273]
 [166.61156]
 [168.0649 ]
 [170.15775]
 [168.22617]
 [168.54857]
 [174.15726]
 [176.06337]
 [177.9595 ]
 [172.7216 ]
 [169.8363 ]
 [166.7732 ]
 [170.31839]
 [170.63953]
 [170.63953]
 [177.01274]
 [176.85469]
 [175.90489]
 [178.11705]
 [181.40709]
 [182.6508 ]
 [177.17072]
 [173.04105]
 [168.0649 ]
 [171.44144]
 [171.92192]
 [169.35376]
 [176.85469]
 [175.58775]
 [176.69656]
 [181.40709]
 [187.26236]
 [185.88797]
 [180.627  ]
 [175.90489]
 [171.76181]
 [175.90489]
 [177.9595 ]
 [176.53838]
 [181.87415]
 [182.18507]
 [182.34041]

In [12]:
#Calculate RMSE
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print("Train RMSE", trainScore)
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
print("Test RMSE", testScore)

Train RMSE 72.849261308701
Test RMSE 223.260886933411
